In [1]:
import re
import gensim 
import mglearn
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import konlpy
import ckonlpy
from konlpy.utils import pprint
from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from konlpy.tag import Kkma, Mecab
import pyLDAvis.gensim
import pyLDAvis.sklearn
import collections
from gensim.models.word2vec import Word2Vec
import copy
import soynlp
import time
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
import networkx as nx
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothicOTF')

In [2]:
class NaverNewsAnalyzer():

        def __init__ (self):
            self.idx_result_df = pd.DataFrame()

            input_file_name = "./result/indexing.txt"

            self.idx_result_df = pd.read_csv(input_file_name,encoding="UTF8",header=None)
            self.filtered_idx_result_df = pd.DataFrame()
            self.idx_result_df.columns = ['date','title','press','content_url_list','file_list']

            self.climate_words = ['기후변화','가뭄', '강추위', '결빙', '그린란드빙하', '기후', '남극빙하', '녹색성장', '대설', '라니냐',
                                  '무더위', '북극빙하', '사막화', '산성비', '쓰나미', '엘니뇨', '열대야', '열섬', '열파', '온난',
                                  '온실가스', '우박', '이산화탄소', '이상고온', '이상기온', '이상저온', '장마', '적설', '집중강우',
                                  '집중호우', '침수', '탄소', '태풍', '파랑', '패랑', '폭설', '폭염', '폭우', '풍량', '풍수해', '한파',
                                  '해수면', '해일', '혹서', '혹한', '홍수', '황사비', '히말라야빙하']

            self.climate_words = "|".join(self.climate_words)
            self.kkma = Kkma()
            self.twitter = ckonlpy.tag.Twitter()
            self.twitter2 = konlpy.tag.Twitter()
            
            self.document_topics = None
            self.topic_words = None
            self.filtered_sentences = None
            self.morph_analyzer = None
            self.w2v_model = None
            self.keyword_list = None
            self.topic_list = None
            self.corpus = None
            self.nouns  = None
            self.temp = None
            
            # 결과저장 디렉토리 생성
            dirname = 'result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './analysis_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './lda_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')

            dirname = './doc_trend_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './keyword_frequency_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
            
            dirname = './keyword_trend_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './lda_stat'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')    
                
            dirname = './lda_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './sentence_level_cowords'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
            
            dirname = './doc_level_cowords'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './w2v_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')
                
            dirname = './filtered_sentence_result'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')    
                
            dirname = './press_statistic_result/'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')  
                
            dirname = './filtered_idx_result/'
            if ((os.path.isdir('./' + dirname + '/')) == False):
                os.mkdir('./' + dirname + '/')     
            
                
            # 네이버 추가 전처리    
            self.idx_result_df['date'] = list(map(str, self.idx_result_df['date']))
            self.idx_result_df['file_list'] = list(map(lambda x: os.path.dirname(input_file_name) + "/" + os.path.basename(x), self.idx_result_df['file_list'].tolist()))

            self.filtered_idx_result_df =  self.idx_result_df
            
            # remove duplicate
            self.remove_duplicate()
            
        def remove_duplicate(self):
            self.filtered_idx_result_df = self.filtered_idx_result_df.drop_duplicates(subset=['title','date'],keep="last")
            self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
        
        def press_statistics(self, norm=False, prefix=""):
            
            # 언론사 추출
            press_list = set(self.filtered_idx_result_df['press'].tolist())
            # 날짜 추출
            
            result_df = pd.DataFrame(columns=press_list,index=['2005','2006','2007','2008','2009','2010','2011','2012',
                                                               '2013','2014','2015','2016','2017'])
            
            x = []
            for press in press_list:
                print(press)
                press_idx = self.filtered_idx_result_df['press'] == press
                temp_filtered_idx_result_df = self.filtered_idx_result_df.loc[press_idx]
                
                dt_list = temp_filtered_idx_result_df['date'].tolist()
                dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], dt_list)))

                x = list(dt_cnt.keys())
                print(x)
                y = list(dt_cnt.values())
                print(y)

                result_df[press][x] = y
        
            result_df = result_df.transpose()
            result_df = result_df.fillna(0)
            result_df.to_csv("./press_statistic_result/" + prefix +  "press_trends.csv",encoding="UTF8")
                
        # 날짜 필터링
        def filtering_date(self,start_date = None, end_date = None):
            dt_index = pd.date_range(start=start_date, end = end_date)
            dt_list = dt_index.strftime("%Y-%m-%d").tolist()
            idx = list(map(lambda x: x in dt_list, self.filtered_idx_result_df['date'].tolist()))
            self.filtered_idx_result_df = self.filtered_idx_result_df.iloc[idx]
            self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
            #print(self.idx_result_df[self.idx_result_df['date'] in dt_list]) 
                        
        # 문서 필터링
        # is_del = True : 제거 필터링 / is_del = False : 포함 필터링
        def filtering_contents(self,rexp,is_del=False):
            file_list =  self.filtered_idx_result_df['file_list'].tolist()
            #print(file_list)
            fidx = []
            didx = []
            for i in range(len(file_list)):
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                f.close()
                p = re.compile(rexp)
                ridx = p.search(doc)
                    
                if(ridx == None):
                   #print("없음")
                    didx.append(i)
                
                else:
                    fidx.append(i)
                
            #print(fidx)
            if is_del == False:
                #rint(fidx)
                self.filtered_idx_result_df = self.filtered_idx_result_df.iloc[fidx]
                self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
            else :
                self.filtered_idx_result_df = self.filtered_idx_result_df.iloc[didx]
                self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
            #print(self.idx_result_df)
        

        # 필터링 된 결과 저장 
            
        def load_lda_result(self,prefix=""):
            self.document_topics = pd.read_csv("./lda_result/" + prefix + "document_topic.csv",encoding="UdTF8",header=None)
            self.topic_words = pd.read_csv("./lda_result/" + prefix + "topic_words.csv",encoding="UTF8",header=None)
                            
            self.topic_list = np.argmax(np.array(self.document_topics),axis=1).tolist()

            
        # 두개의 인덱스 파일 결합   
        def combine_idx_result(self, prev):
            #print("Current")
            #print(self.filtered_idx_result_df)
            #print("Prev Index result")
            #print(prev.filtered_idx_result_df)
            self.filtered_idx_result_df = pd.concat([self.filtered_idx_result_df,prev.filtered_idx_result_df])
            self.filtered_idx_result_df = self.filtered_idx_result_df.drop_duplicates(['date','title'])
            self.filtered_idx_result_df = self.filtered_idx_result_df.sort_values(["file_list"], ascending=True)
            self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
            
        # 문서 트랜드 분석
        def doc_trend_analysis(self, norm=False, prefix="",title="네이버 환경뉴스",xlab="년도",ylab="문서수"):
            
            # 날짜 추출
            dt_list = self.filtered_idx_result_df['date'].tolist()
            dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], dt_list)))
         
            x = list(dt_cnt.keys())
            y = list(dt_cnt.values())
            
            # norm= True (정규화, 상대적 비교)
            if(norm == True) :
                prefix = prefix + "norm_"
                dt_index = pd.date_range(start=min(dt_list), end = max(dt_list))
                all_dt_list = dt_index.strftime("%Y-%m-%d").tolist()
                idx = list(map(lambda x: x in all_dt_list, self.idx_result_df['date'].tolist()))
                temp_df = self.idx_result_df.iloc[idx]
                temp_df = temp_df.reset_index(drop=True)
                temp_df_list = temp_df['date'].tolist()
                #print(temp_df)

                all_dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], temp_df_list)))
                all_y = list(all_dt_cnt.values())

                y =  np.array(y) / np.array(all_y)
                y =  y.tolist()

            plt.xticks(rotation=50)
            plt.plot(x,y,c="b", lw=2, ls="--", marker="o", ms=10, mec="g", mew=2, mfc="r")
            plt.title(title)
            plt.xlabel(xlab)
            plt.ylabel(ylab)
            plt.savefig("./doc_trend_result/" + prefix + "doc_trends.png",dpi=200)

            result = pd.DataFrame([x,y]).transpose()
            result.to_csv("./doc_trend_result/" + prefix + "doc_trends.csv",encoding="UTF8")
            
        # 키워드 빈도수 체크
        def keyword_frequency_analysis(self,num=100,is_tfidf = True,is_noun=True, ma="twitter2",prefix="",s_words=[]):
            
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2    
                
            file_list = self.filtered_idx_result_df['file_list'].tolist()
            print(len(file_list))
            print("Keyword Analysis")
            corpus = [""] * len(file_list)
      
            # 파일 내용 읽어서 코퍼스에 저장
            process_time = time.time()
            for i in range(len(file_list)):
                if(i==1000):
                    process_time = time.time() - process_time
                    print(process_time)
                print(i)
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                corpus[i] = " ".join(self.morph_analyzer.nouns(doc))
                
            f.close()
          
            # Stop words 
            stop_words = ['광화문','기념','포항','화제','전북','남해','전남','유행','일보','마리','안전','오후','버스',
              '관리','세상','경남','업계','하늘','우산','종로구','하경민','미디어','중부','기자회견','출근',
              '공동','활짝','오전','남부','기술','청주','기사','진주','추진','날씨','집결','계획','대표',
              '밀양시','장관','클릭','공사','전체','판단','전망','함양','관광객','여러분','영상','문의',
              '지정','대체로','경향','억원','경북','지난','대회','제주도','공감','포항시','의원',
              '사진','춘천','군청','평가','산천어','한국','한윤','스님','축제','기상청','최저','추위','동해',
              '규모','행사','적극','함양군','호기','대책','경기','울산','기업','디지털','신문','일본',
              '관계자','최고','코리아','피해','예상','시설','지구','미만','종원','복원','최신','미리','발견',
              '회원','지방','예보','부산','제보','화천','만화','관련','총회','현상','성장','홍보','우리',
              '사업','금지','기준','트렌드','광주','연구','환경','물결','제품','제공','무료','작품','참석자',
              '제주','평년','강원','국제','가치나','가운데','농도','처리','연합','정책','세계','경우',
              '조사','공단','대한','모바일','밀양','공원','지역','국내','주변','서울시','국회','전재',
              '경기도','국가','강릉','위원회','정부','전주','올해','협의','초과','산업','차량','대강','언론',
              '연합뉴스','인근','단체','발령','북부','정보','무단','조성','김용만','대해','아침','대구',
              '결과','태안','안지','환경부','협악','먹이','전국','보이','지원','경향신문','주의보','건설', 
              '경제','자웅','위해','채용','배포','발효','독자','조금','개발','때문','일원','꽃망울','회의',
              '차용','마을','수산','기상','협회','해수욕장','유엔','여수','사회','단지','유치','검사','구간',
              '파란','바람','서해','해상','전시','창원','기름','새만금','반대','골프장','중단','동해안',
              '장미','제도','인증','미투데이','바닷가','설악산','한라산','아파트','해경','충남','월드컵',
              '완연','터트린','맑은','피어','만끽','청명','수준','소방','북상','영동','풍랑','영산강','환경청',
              '일대','정비','꽃샘추위','계정','소통','미국','사람','한겨레','상승','보고서','투표','이자',
              '여의도','윤중로','남산','설천면','서면','밀양아리랑','대전','충북','최강','주가','빅데이터',
              '김선웅','이영환','학생','권역','춘분','스타벅스','커피','청계천','태백산','배훈식','김진아',
              '페이스북','바로가기','트위터','몽골','아들','해제','처분','기해','안성','환자','평균','재판',
              '캠페인','배출','사용','점검','채널','분리','설정','활동','영향','보통','이상','측정','상태', 
              '정취','낙동강','해체','환경운동','인사말','공급','원인','족구','근본','진도','계속','외모',
              '업체','회관','상림','오늘','내일','시세','연구원','지난해','센터','대상','이번','사월','예정', 
              '가장','침대','타워','아래','종록','재촉','시간','파크','대저','스포츠','터트려','농촌진흥청',
              '통해' '차관','이데일리','동물원','이천','경보','소독','인양','가을','장마전선','봄비','겨울',
              '김은경','안병옥','이정선','여신','래시','명동','퍼포먼스','경찰','성분','영서','차차','빙어',
              '세종','인천','함양읍','수원','서초구','중구','송파구','거창군','용인','남면','삼문동','매화',
              '이나','통해','에서','잡고','맺힌','뽐내','채팅','맨손','잡기','평창동계올림픽','긴급','유채꽃',
              '구매','현장','구독','생활','류형','포토','메인','단속','소식','공항','제주시','개막','물질',
              '인제','남구','태백','나들이','강진','김태식','동구','화천군','낚시','우장','속초','사육',
              '개선','시행','문제','차관','방안','주무관','무술년','새해','상춘객','공모전','현재','의성',
              '화천읍','낭만','비상','뿌옇','출근길','도로','대부분','앞바다','조치','매우','고성','차바',
              '새벽','밤새','일부','전날','군산','중심','맑음','주말','보가','임태훈','기록','가족','박주성',
              '체험','청사','발생','산림청','설치','최근','협력','대응','날인','자태','인제군','마지막',
              '식목일','서귀포시','영등포구','성남시','고승민','발걸음','고범준','부제','치가','사거리','거리',
              '롯데','주위','모습','월드','용산구','망신','이하늬','세영','야구','콜라','가드','광장',
              '유혹','첫날','탐방','자락','세종시','맞이','패럴림픽','하루','이동면','경칩','성남','입춘',
              '기승','보기','협약','기관','운영','수거','작업','조명','장병','주차장','올림픽','운항','빗방울',
              '백승','곳곳','방역','하동','하동군','연휴','추석','황금','배병수','홍천','평창','과장',
              '휴식','강릉시','유형','화창','동진','최진','사이','포럼','개최','조경규','국립',
              '운세','후회','여기','직원','물든','대통령','호텔','바닥','북구','청와대','추연','유해',
              '왼쪽','시장','공연','주제','오른쪽','수원시','촬영','주년','정원','청소','현지','여명','강남구',
              '판매','통합','종합','네이버','지청','당분간','추상','햇살','연일','풍경','흥부가','위로',
              '이틀','한경닷컴','유승민','무소속','조현아','노출','총선','스폰서','비키니','치어리더',
              '수확','개방','상황','본부','관측','스탠드','당부','푸른','정상','김경','고현면','고현면',
              '오색','정유년','저녁','서울대','훈련','채취','권현구','김인철','이정섭','미녀','몸짱','에버랜드',
              '매일','서비스','최창호','피해자','병원','저작권','적발','사고','남남서','건물','신고','국제공항',
              '교육청','행동','행동','데이트','확률','가끔','점차','전해','강원도','일이','핫이슈','봄꽃',
              '오픈','서비스','윤성규','성매매','과천','대한항공','몸매','동영상','봉사활동','유강','정선',
              '후보','문호','업데이트','고속도로','태양','물놀이','의상','쯔위','드레스','학교','현황',
              '남남서','동해','내륙','중부지방','분야','폭스바겐','문화','광장','주연','벗꽃','리우','지사',
              '경주','경주시','글로벌','실시간','인턴','후보','특별','형산강','연일읍','북한','마트','가동',
              '한경닷컴','박기량','교육','인사','빗물','안양','그룹','농림축산식품부','이상인','대기',
              '교수','내년','운동장','사막','내외','가장자리','여객선','호인','누드','확산','파문','자료',
              '아나운서','산간','절기','만원','수도권','강수','기상대','주의','람사르','지리산','발전',
              '결항','산수유','북주','외국인','인산인해','주상','부평','둘째','서울시장','지하철','공공기관',
              '자자체','촉구','동남아','새끼','학원','산지','인파','썰매','사무소','휴일','로봇','금은',
              '우리나라','적용','권현구','가정','연휴','나무','설경','시청','나흘','우수','터널','목련',
              '일출','개나리','기운','스카이','연일','성동구','산불','진화','낙시터','평택','매장','대란',
              '동아일보','공기','라며','산책','강종민','축구','프레','심기','작용','카메라','사랑','부처님',
              '송골송골','이물질','개양귀비','구경','바다','시민활동','동급','튤립','해양','모내기','안개',
              '경기장','둥지','지방선거','철쭉','탄천','드론','서귀포','비행','소만','추억','다리','고기압',
              '대진','고송민','거부','활약','시민행동','선언','이의','취소','형형색색','무상','잠실','운행',
              '대기오염','임현정','협약','전망대','한강','식용','군락','환경재단','어린이집','구름','울릉도',
              '연꽃','밀양강','전문가','모두','코스모스','마포구','봄날','대박','중앙','배경','꽁꽁','단풍',
              '눈길','기온','영하','분수','계곡','농장','화성','교양','가득','먼지','단계','고양','고양시',
              '화단','이형기','알피','대학교','성큼','선선','참가자','태현','광진구','평창군','최동준',
              '처서','동민','서툴','해운대해수욕장','해운대','극심','본격','물고기','니스','대관령','상공',
              '전신','서해안','길이','나들','소재','시내','독도','동선','인공','산란','공개','창원시',
              '민주당','이진욱','방문','노동','물이','삼척','구조','통행','밤사이','안산','뉴시스',
              '서울','벚꽃','뉴스','시민','기자','주민','국민','지속','지자체','북적','불법','최대',
              '두번째','세종로','질환','이용','보고','박원순','뿌연','유튜브','모든','주시 반영','주시',
              '반영','아름','더위','기고','아름 다운','도심','다운','국적','고명진','어가','지고','지나',
              '어진','이루','알리','상암동','대공','시작','연출','부터','면서','부터','유의','다가',
              '면서','린다','만들기','마련','전시회','다소','거나','아지','나타','제거','총장','참석',
              '발길','화려','라보','피하','가운','사흠','국적','매달','연출','대공','기고','반영','부터 아지',
              '다가 부터','부터 부터','수요일','화요일','월요일','목요일','금요일','토요일','일요일','유의 바란',
              '박원순','온라인','공식','신청','진행','활용','정도','설명','기간','이후','다른',
              '발표','주장','지적','우려','가능성','정도','요구','과정','한편','동안','다시','상부',
              '이어진','온라인','공식','우려','처음','이유','다시','정도','요구','크게','사실',
              '기간','포함','다음','화보','한편','가능성','이후','설명','분석','지금','각각','발표',
              '크게','동안','이후','처음','주요','내용','실시','어린이','확인','특보','비롯','분포',
              '증가','중인','가량','아시아','이동','지역별','규칙','바로','이하'
             ]
            
            stop_words = list(set(stop_words))
            if(len(s_words) != 0):
                stop_words = s_words
            
            vect = CountVectorizer(stop_words = stop_words,ngram_range=(1, 2),min_df=0.005,max_df=0.3)
            #ocab = vectorizer.get_feature_names()
            #print("Execution")
            X = vect.fit_transform(corpus)
            if(is_tfidf == True):
                X = TfidfTransformer().fit_transform(X)
                prefix = prefix + "norm_"
                
            count = X.toarray().sum(axis=0)
            idx = np.argsort(-count)
            count = count[idx]

            feature_name = np.array(vect.get_feature_names())[idx]
            #plt.bar(range(len(count)), count)
            #plt.show()

            self.keyword_list = list(zip(feature_name[:num], count[:num]))
            result = pd.DataFrame(self.keyword_list)
            result.to_csv("./keyword_frequency_result/" + prefix + "keyword_frequency.csv",encoding="UTF8")            
            
        # 키워드 트랜드 분석
        # norm = 년도별 정규화
        # voca = "분석할 단어 설정"
        def keyword_trend_analysis(self, prefix="", norm=False, ma ='twitter2',voca = ""):
            
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2
            
            temp_df = self.filtered_idx_result_df
            file_list = self.filtered_idx_result_df['file_list'].tolist()
            
            x = list()
            
            # 각 vocabulary에 대해서 수행
            result = pd.DataFrame()
            for v in voca:
                #print(v)
                fidx = []
                didx = []
                for i in range(len(file_list)):
                    f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                    doc = f.read()
                    f.close()
                    p = re.compile(v)
                    ridx = p.search(doc)
                    if(ridx == None):
                        didx.append(i)
                    else:
                        fidx.append(i)
                    
                # 날짜 추출
                dt_list = [self.filtered_idx_result_df['date'].tolist()[sidx] for sidx in fidx]
                dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], dt_list)))
            
                x = list(dt_cnt.keys())
                y = list(dt_cnt.values())
                y = pd.DataFrame(y)
                result = pd.concat([result,y],axis=1)
                
           # dt_list = self.filtered_idx_result_df['date'].tolist()   
           # dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], dt_list)))        
            
            result.columns = voca

            if(norm == True):
                prefix = "norm_" + prefix
                temp_df_list = self.filtered_idx_result_df['date'].tolist()
                all_dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], temp_df_list)))
                
                all_y = list(all_dt_cnt.values())
                for i in range(len(all_y)):
                    #print(result)
                    result.iloc[i,:] = result.iloc[i,:] / all_y[i]
            
            x = pd.DataFrame(x)
            result = pd.concat([x,result],axis=1)
                        
            result.to_csv("./keyword_trend_result/" + prefix + "keyword_trends.csv",encoding="UTF8")
    
        # 토픽 모델링
        def topic_modeling(self,n_topics, ma = "twitter2", prefix = "",random_seed=1,s_words=[]):
            
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2
                       
            # 형태소 분석을 기본적으로 수행
            file_list = self.filtered_idx_result_df['file_list'].tolist()
            #print(file_list)
            corpus = [""] * len(file_list)

            # 파일 내용 읽어서 코퍼스에 저장

            for i in range(len(file_list)):
                #print(i)
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                f.close()
                corpus[i] = " ".join(self.morph_analyzer.nouns(doc))

            #print(corpus)

            # Stop words ,
     
            stop_words = ['광화문','기념','포항','화제','전북','남해','전남','유행','일보','마리','안전','오후','버스',
              '관리','세상','경남','업계','하늘','우산','종로구','하경민','미디어','중부','기자회견','출근',
              '공동','활짝','오전','남부','기술','청주','기사','진주','추진','날씨','집결','계획','대표',
              '밀양시','장관','클릭','공사','전체','판단','전망','함양','관광객','여러분','영상','문의',
              '지정','대체로','녹색','경향','억원','경북','지난','대회','제주도','공감','포항시','의원',
              '사진','춘천','군청','평가','산천어','한국','한윤','스님','축제','기상청','최저','추위','동해',
              '규모','행사','적극','함양군','호기','대책','경기','울산','기업','디지털','신문','일본',
              '관계자','최고','코리아','피해','예상','시설','지구','미만','종원','복원','최신','미리','발견',
              '회원','지방','예보','부산','제보','화천','만화','관련','총회','현상','성장','홍보','우리',
              '사업','금지','기준','트렌드','광주','연구','환경','물결','제품','제공','무료','작품','참석자',
              '제주','평년','강원','국제','가치나','가운데','농도','처리','연합','정책','세계','경우',
              '조사','공단','대한','모바일','밀양','공원','지역','국내','주변','서울시','국회','전재',
              '경기도','국가','강릉','위원회','정부','전주','올해','협의','초과','산업','차량','대강','언론',
              '연합뉴스','인근','단체','발령','북부','정보','무단','조성','김용만','대해','아침','대구',
              '결과','태안','안지','환경부','협악','먹이','전국','보이','지원','경향신문','주의보','건설', 
              '경제','자웅','위해','채용','배포','발효','독자','조금','개발','때문','일원','꽃망울','회의',
              '차용','마을','수산','기상','협회','해수욕장','유엔','여수','사회','단지','유치','검사','구간',
              '파란','바람','서해','해상','전시','창원','기름','새만금','반대','골프장','중단','동해안',
              '장미','제도','인증','미투데이','바닷가','설악산','한라산','아파트','해경','충남','월드컵',
              '완연','터트린','맑은','피어','만끽','청명','수준','소방','북상','영동','풍랑','영산강','환경청',
              '일대','정비','꽃샘추위','계정','소통','미국','사람','한겨레','상승','보고서','투표','이자',
              '여의도','윤중로','남산','설천면','서면','밀양아리랑','대전','충북','최강','주가','빅데이터',
              '김선웅','이영환','학생','권역','춘분','스타벅스','커피','청계천','태백산','배훈식','김진아',
              '페이스북','바로가기','트위터','몽골','아들','해제','처분','기해','안성','환자','평균','재판',
              '캠페인','배출','사용','점검','채널','분리','설정','활동','영향','보통','이상','측정','상태', 
              '정취','낙동강','해체','환경운동','인사말','공급','원인','족구','근본','진도','계속','외모',
              '업체','회관','상림','오늘','내일','시세','연구원','지난해','센터','대상','이번','사월','예정', 
              '가장','침대','타워','아래','종록','재촉','시간','파크','대저','스포츠','터트려','농촌진흥청',
              '통해' '차관','이데일리','동물원','이천','경보','소독','인양','가을','장마전선','봄비','겨울',
              '김은경','안병옥','이정선','여신','래시','명동','퍼포먼스','경찰','성분','영서','차차','빙어',
              '세종','인천','함양읍','수원','서초구','중구','송파구','거창군','용인','남면','삼문동','매화',
              '이나','통해','에서','잡고','맺힌','뽐내','채팅','맨손','잡기','평창동계올림픽','긴급','유채꽃',
              '구매','현장','구독','생활','류형','포토','메인','단속','소식','공항','제주시','개막','물질',
              '인제','남구','태백','나들이','강진','김태식','동구','화천군','낚시','우장','속초','사육',
              '개선','시행','문제','차관','방안','주무관','무술년','새해','상춘객','공모전','현재','의성',
              '화천읍','낭만','비상','뿌옇','출근길','도로','대부분','앞바다','조치','매우','고성','차바',
              '새벽','밤새','일부','전날','군산','중심','맑음','주말','보가','임태훈','기록','가족','박주성',
              '체험','청사','발생','산림청','설치','최근','협력','대응','날인','자태','인제군','마지막',
              '식목일','서귀포시','영등포구','성남시','고승민','발걸음','고범준','부제','치가','사거리','거리',
              '롯데','주위','모습','월드','용산구','망신','이하늬','세영','야구','콜라','가드','광장',
              '유혹','첫날','탐방','자락','세종시','맞이','패럴림픽','하루','이동면','경칩','성남','입춘',
              '기승','보기','협약','기관','운영','수거','작업','조명','장병','주차장','올림픽','운항','빗방울',
              '백승','곳곳','방역','하동','하동군','연휴','추석','황금','배병수','홍천','평창','과장',
              '휴식','강릉시','유형','화창','동진','최진','사이','포럼','개최','조경규','국립',
              '운세','후회','여기','직원','물든','대통령','호텔','바닥','북구','청와대','추연','유해',
              '왼쪽','시장','공연','주제','오른쪽','수원시','촬영','주년','정원','청소','현지','여명','강남구',
              '판매','통합','종합','네이버','지청','당분간','추상','햇살','연일','풍경','흥부가','위로',
              '이틀','한경닷컴','유승민','무소속','조현아','노출','총선','스폰서','비키니','치어리더',
              '수확','개방','상황','본부','관측','스탠드','당부','푸른','정상','김경','고현면','고현면',
              '오색','정유년','저녁','서울대','훈련','채취','권현구','김인철','이정섭','미녀','몸짱','에버랜드',
              '매일','서비스','최창호','피해자','병원','저작권','적발','사고','남남서','건물','신고','국제공항',
              '교육청','행동','행동','데이트','확률','가끔','점차','전해','강원도','일이','핫이슈w','봄꽃',
              '오픈','서비스','윤성규','성매매','과천','대한항공','몸매','동영상','봉사활동','유강','정선',
              '후보','문호','업데이트','고속도로','태양','물놀이','의상','쯔위','드레스','학교','현황',
              '남남서','동해','내륙','중부지방','분야','폭스바겐','문화','광장','주연','벗꽃','리우','지사',
              '경주','경주시','글로벌','실시간','인턴','후보','특별','형산강','연일읍','북한','마트','가동',
              '한경닷컴','박기량','교육','인사','빗물','안양','그룹','농림축산식품부','이상인','대기',
              '교수','내년','운동장','사막','내외','가장자리','여객선','호인','누드','확산','파문','자료',
              '아나운서','산간','절기','만원','수도권','강수','기상대','주의','람사르','지리산','발전',
              '결항','산수유','북주','외국인','인산인해','주상','부평','둘째','서울시장','지하철','공공기관',
              '자자체','촉구','동남아','새끼','학원','산지','인파','썰매','사무소','휴일','로봇','금은',
              '우리나라','적용','권현구','가정','연휴','나무','설경','시청','나흘','우수','터널','목련',
              '일출','개나리','기운','스카이','연일','성동구','산불','진화','낙시터','평택','매장','대란',
              '동아일보','공기','라며','산책','강종민','축구','프레','심기','작용','카메라','사랑','부처님',
              '송골송골','이물질','개양귀비','구경','바다','시민활동','동급','튤립','해양','모내기','안개',
              '경기장','둥지','지방선거','철쭉','탄천','드론','서귀포','비행','소만','추억','다리','고기압',
              '대진','고송민','거부','활약','시민행동','선언','이의','취소','형형색색','무상','잠실','운행',
              '대기오염','임현정','협약','전망대','한강','식용','군락','환경재단','어린이집','구름','울릉도',
              '연꽃','밀양강','전문가','모두','코스모스','마포구','봄날','대박','중앙','배경','꽁꽁','단풍',
              '눈길','기온','영하','분수','계곡','농장','화성','교양','가득','먼지','단계','고양','고양시',
              '화단','이형기','알피','대학교','성큼','선선','참가자','태현','광진구','평창군','최동준',
              '처서','동민','서툴','해운대해수욕장','해운대','극심','본격','물고기','니스','대관령','상공',
              '전신','서해안','길이','나들','소재','시내','독도','동선','인공','산란','공개','창원시',
              '민주당','이진욱','방문','노동','물이','삼척','구조','통행','밤사이','안산','뉴시스',
              '서울','벚꽃','뉴스','시민','기자','주민','국민','지속','지자체','북적','불법','최대',
              '두번째','세종로','질환','이용','보고','박원순','뿌연','유튜브','모든','주시 반영','주시',
              '반영','아름','더위','기고','아름 다운','도심','다운','국적','고명진','어가','지고','지나',
              '어진','이루','알리','상암동','대공','시작','연출','부터','면서','부터','유의','다가',
              '면서','린다','만들기','마련','전시회','다소','거나','아지','나타','제거','총장','참석',
              '발길','화려','라보','피하','가운','사흠','국적','매달','연출','대공','기고','반영','부터 아지',
              '다가 부터','부터 부터','수요일','화요일','월요일','목요일','금요일','토요일','일요일','유의 바란',
              '박원순','온라인','공식','신청','진행','활용','정도','설명','기간','이후','다른',
              '발표','주장','지적','우려','가능성','정도','요구','과정','한편','동안','다시','상부',
              '이어진','온라인','공식','우려','처음','이유','다시','정도','요구','크게','사실',
              '기간','포함','다음','화보','한편','가능성','이후','설명','분석','지금','각각','발표',
              '크게','동안','이후','처음','주요','내용','실시'                 
             ]
        
            stop_words = list(set(stop_words))
            if(len(s_words) != 0):
                stop_words = s_words
            
            vect = CountVectorizer(stop_words = stop_words,ngram_range=(1, 2),min_df=0.005,max_df=0.3)
            X = vect.fit_transform(corpus)
            X = TfidfTransformer().fit_transform(X)

            #vect = CountVectorizer(stop_words = stop_words,min_df=0.005,max_df=0.3)

            n_words = 300
            for i in n_topics:
                print(i)
                # LDA 수횅
                lda = LatentDirichletAllocation(n_components = i,learning_method="batch",random_state=random_seed)
                #lda = LatentDirichletAllocation(learning_method="batch")
                self.document_topics = lda.fit_transform(X)
                
                sorting = np.argsort(lda.components_, axis=1)[:,::-1]
                feature_names = np.array(vect.get_feature_names())
                
                #mglearn.tools.print_topics(topics=range(i), feature_names = feature_names, sorting=sorting, topics_per_chunk=20, n_words=20)
                #self.filtered_idx_result_df["topic"] = np.argmax(self.document_topics,axis=1).tolist()
                
                self.topic_list = np.argmax(self.document_topics,axis=1).tolist()
                
                self.topic_words = pd.DataFrame(feature_names[sorting])
                self.topic_words = self.topic_words.T
                self.topic_words = self.topic_words[:n_words]
                
                # 파일이름 지정
                filename =  "./lda_result/" +  str(prefix) + "_result_n_" + str(i) + "_" 

                # 파일 저장
                self.document_topics = pd.DataFrame(self.document_topics)
                self.document_topics.to_csv(filename + "document_topic.csv",header=None,index=False,encoding="UTF8")
                self.topic_words.to_csv(filename + "topic_words.csv",header=None,index=False,encoding="UTF8")
                
                # 시각화 및 저장
                self.topic_modeling_vis(lda, X, vect, filename)

        def topic_modeling_vis(self,lda,dtm,vect,filename):
            pyldavis = pyLDAvis.sklearn.prepare(lda, dtm,vect)
            pyLDAvis.save_html(pyldavis,filename + "lda.html")
            
        def select_topics(self,topics):
            #print("Select Topics")
            topic_list = self.topic_list
            #print(topic_list)
            tidx = list(map(lambda x: x in topics , topic_list))
            #print(tidx)
            
            self.filtered_idx_result_df = self.filtered_idx_result_df.iloc[tidx,:]
            self.filtered_idx_result_df = self.filtered_idx_result_df.reset_index(drop=True)
            self.topic_list = np.array(self.topic_list)[tidx].tolist()
            
        def topic_modeling_statistics(self,prefix=""):
            
            topic_list = self.topic_list
            result = pd.DataFrame()

            dt = None
            for t in range(max(topic_list) + 1):
                print(t)
                sub_idx = (np.array(topic_list) == t)
                sub_idx = list(sub_idx)
                sub_result = self.filtered_idx_result_df.loc[sub_idx,:]
                date_list = sub_result['date']
                all_dt_cnt = collections.Counter(list(map(lambda x : x.split("-")[0], date_list)))
                all_y = list(all_dt_cnt.values())
                all_y = pd.DataFrame(all_y)
                result = pd.concat([result,all_y],axis=1)            

            dt = list(all_dt_cnt.keys())
            dt = pd.DataFrame(dt)
            result = pd.concat([dt,result],axis=1)

            result.to_csv("./lda_stat/" + prefix + "lda_stat.csv",encoding="UTF8")
        
            
        def extract_sentence(self,rexp,ma="twitter2",prefix=""):
            w1 = rexp
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2  

            file_list =  self.filtered_idx_result_df['file_list'].tolist()
            filtered_sentences= []
            for i in range(len(file_list)):
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                f.close()
                sentences = self.split_text(doc)
                
                for s in sentences:
                    filtered_sentences.append(s)
            
            self.temp = pd.DataFrame(filtered_sentences)
            
            for w in w1:
                temp = []
                for s in filtered_sentences :
                    p = re.compile(w)
                    ridx = p.search(s)

                    if(ridx == None):
                       #print("없음")
                       continue
                        
                    s = s.strip()
                    #s = re.sub('[^가-힝0-9a-zA-Z\\s]', '', s)
                    temp.append(s)
                filtered_sentences = temp
            
            self.filtered_sentences = filtered_sentences
            result = pd.DataFrame(self.filtered_sentences)
            
            result.to_csv("./filtered_sentences_result/" + prefix + "filtered_sentences.csv")
            
        # word2vec 학습
        def training_keyword_w2v(self,ma="twitter2"):
            file_list = self.filtered_idx_result_df['file_list'].tolist()
            corpus = [""] * len(file_list)
            # 형태소 분석기 선택

            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2
                
            # 파일 내용 읽어서 코퍼스에 저장
            for i in range(len(file_list)):
                #print(i)
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                
                doc = re.sub('[^가-힝0-9a-zA-Z\\s]', '', doc)
                corpus[i] = self.morph_analyzer.nouns(doc)

                
                
            model = Word2Vec(corpus)
            model.init_sims(replace=True)
            self.w2v_model = model
        
        # word2vec 학습 결과 저장
        def w2v_analysis(self,word,num=100):
            #print(self.w2v_model.most_similatr(word,topn=num))
            result = pd.DataFrame(self.w2v_model.most_similar(word,topn=num))
            result2 = pd.DataFrame(self.w2v_model.predict_output_word(word, topn=num))
            filename = "".join(word)
            result.to_csv("./w2v_result/" + "w2v_" + filename + ".csv",encoding="UTF8")
            result.to_csv("./w2v_result/" + "w2v2_" + filename + ".csv",encoding="UTF8")
       
        # 문서 레벨 동시 출현 단어
        def doc_level_coccurrence_words(self,w1, num=100,ma="twitter2",prefix="",s_words=[]):
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2  
                
            # 문서추출
            file_list =  self.filtered_idx_result_df['file_list'].tolist()
            temp_idx_result_df = pd.DataFrame()
            #print(file_list)
            
            fidx = []
            didx = []
            for w in w1 :
                for i in range(len(file_list)):
                    f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                    doc = f.read()
                    p = re.compile(w)
                    ridx = p.search(doc)

                    if(ridx == None):
                       #print("없음")
                        didx.append(i)

                    else:
                        fidx.append(i)
                        
            fidx = list(set(fidx))
            #print(fidx)
            temp_idx_result_df = self.filtered_idx_result_df.iloc[fidx]
            temp_idx_result_df = temp_idx_result_df.reset_index(drop=True)
            
            file_list =  temp_idx_result_df['file_list'].tolist()
            
            corpus = []     
            for i in range(len(file_list)):
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                
                # 파일 내용 읽어서 코퍼스에 저장
                corpus.append(" ".join(self.morph_analyzer.nouns(doc)))

                      # Stop words 
            stop_words = ['광화문','기념','포항','화제','전북','남해','전남','유행','일보','마리','안전','오후','버스',
              '관리','세상','경남','업계','하늘','우산','종로구','하경민','미디어','중부','기자회견','출근',
              '공동','활짝','오전','남부','기술','청주','기사','진주','추진','날씨','집결','계획','대표',
              '밀양시','장관','클릭','공사','전체','판단','전망','함양','관광객','여러분','영상','문의',
              '지정','대체로','경향','억원','경북','지난','대회','제주도','공감','포항시','의원',
              '사진','춘천','군청','평가','산천어','한국','한윤','스님','축제','기상청','최저','추위','동해',
              '규모','행사','적극','함양군','호기','대책','경기','울산','기업','디지털','신문','일본',
              '관계자','최고','코리아','피해','예상','시설','지구','미만','종원','복원','최신','미리','발견',
              '회원','지방','예보','부산','제보','화천','만화','관련','총회','현상','성장','홍보','우리',
              '사업','금지','기준','트렌드','광주','연구','환경','물결','제품','제공','무료','작품','참석자',
              '제주','평년','강원','국제','가치나','가운데','농도','처리','연합','정책','세계','경우',
              '조사','공단','대한','모바일','밀양','공원','지역','국내','주변','서울시','국회','전재',
              '경기도','국가','강릉','위원회','정부','전주','올해','협의','초과','산업','차량','대강','언론',
              '연합뉴스','인근','단체','발령','북부','정보','무단','조성','김용만','대해','아침','대구',
              '결과','태안','안지','환경부','협악','먹이','전국','보이','지원','경향신문','주의보','건설', 
              '경제','자웅','위해','채용','배포','발효','독자','조금','개발','때문','일원','꽃망울','회의',
              '차용','마을','수산','기상','협회','해수욕장','유엔','여수','사회','단지','유치','검사','구간',
              '파란','바람','서해','해상','전시','창원','기름','새만금','반대','골프장','중단','동해안',
              '장미','제도','인증','미투데이','바닷가','설악산','한라산','아파트','해경','충남','월드컵',
              '완연','터트린','맑은','피어','만끽','청명','수준','소방','북상','영동','풍랑','영산강','환경청',
              '일대','정비','꽃샘추위','계정','소통','미국','사람','한겨레','상승','보고서','투표','이자',
              '여의도','윤중로','남산','설천면','서면','밀양아리랑','대전','충북','최강','주가','빅데이터',
              '김선웅','이영환','학생','권역','춘분','스타벅스','커피','청계천','태백산','배훈식','김진아',
              '페이스북','바로가기','트위터','몽골','아들','해제','처분','기해','안성','환자','평균','재판',
              '캠페인','배출','사용','점검','채널','분리','설정','활동','영향','보통','이상','측정','상태', 
              '정취','낙동강','해체','환경운동','인사말','공급','원인','족구','근본','진도','계속','외모',
              '업체','회관','상림','오늘','내일','시세','연구원','지난해','센터','대상','이번','사월','예정', 
              '가장','침대','타워','아래','종록','재촉','시간','파크','대저','스포츠','터트려','농촌진흥청',
              '통해' '차관','이데일리','동물원','이천','경보','소독','인양','가을','장마전선','봄비','겨울',
              '김은경','안병옥','이정선','여신','래시','명동','퍼포먼스','경찰','성분','영서','차차','빙어',
              '세종','인천','함양읍','수원','서초구','중구','송파구','거창군','용인','남면','삼문동','매화',
              '이나','통해','에서','잡고','맺힌','뽐내','채팅','맨손','잡기','평창동계올림픽','긴급','유채꽃',
              '구매','현장','구독','생활','류형','포토','메인','단속','소식','공항','제주시','개막','물질',
              '인제','남구','태백','나들이','강진','김태식','동구','화천군','낚시','우장','속초','사육',
              '개선','시행','문제','차관','방안','주무관','무술년','새해','상춘객','공모전','현재','의성',
              '화천읍','낭만','비상','뿌옇','출근길','도로','대부분','앞바다','조치','매우','고성','차바',
              '새벽','밤새','일부','전날','군산','중심','맑음','주말','보가','임태훈','기록','가족','박주성',
              '체험','청사','발생','산림청','설치','최근','협력','대응','날인','자태','인제군','마지막',
              '식목일','서귀포시','영등포구','성남시','고승민','발걸음','고범준','부제','치가','사거리','거리',
              '롯데','주위','모습','월드','용산구','망신','이하늬','세영','야구','콜라','가드','광장',
              '유혹','첫날','탐방','자락','세종시','맞이','패럴림픽','하루','이동면','경칩','성남','입춘',
              '기승','보기','협약','기관','운영','수거','작업','조명','장병','주차장','올림픽','운항','빗방울',
              '백승','곳곳','방역','하동','하동군','연휴','추석','황금','배병수','홍천','평창','과장',
              '휴식','강릉시','유형','화창','동진','최진','사이','포럼','개최','조경규','국립',
              '운세','후회','여기','직원','물든','대통령','호텔','바닥','북구','청와대','추연','유해',
              '왼쪽','시장','공연','주제','오른쪽','수원시','촬영','주년','정원','청소','현지','여명','강남구',
              '판매','통합','종합','네이버','지청','당분간','추상','햇살','연일','풍경','흥부가','위로',
              '이틀','한경닷컴','유승민','무소속','조현아','노출','총선','스폰서','비키니','치어리더',
              '수확','개방','상황','본부','관측','스탠드','당부','푸른','정상','김경','고현면','고현면',
              '오색','정유년','저녁','서울대','훈련','채취','권현구','김인철','이정섭','미녀','몸짱','에버랜드',
              '매일','서비스','최창호','피해자','병원','저작권','적발','사고','남남서','건물','신고','국제공항',
              '교육청','행동','행동','데이트','확률','가끔','점차','전해','강원도','일이','핫이슈','봄꽃',
              '오픈','서비스','윤성규','성매매','과천','대한항공','몸매','동영상','봉사활동','유강','정선',
              '후보','문호','업데이트','고속도로','태양','물놀이','의상','쯔위','드레스','학교','현황',
              '남남서','동해','내륙','중부지방','분야','폭스바겐','문화','광장','주연','벗꽃','리우','지사',
              '경주','경주시','글로벌','실시간','인턴','후보','특별','형산강','연일읍','북한','마트','가동',
              '한경닷컴','박기량','교육','인사','빗물','안양','그룹','농림축산식품부','이상인','대기',
              '교수','내년','운동장','사막','내외','가장자리','여객선','호인','누드','확산','파문','자료',
              '아나운서','산간','절기','만원','수도권','강수','기상대','주의','람사르','지리산','발전',
              '결항','산수유','북주','외국인','인산인해','주상','부평','둘째','서울시장','지하철','공공기관',
              '자자체','촉구','동남아','새끼','학원','산지','인파','썰매','사무소','휴일','로봇','금은',
              '우리나라','적용','권현구','가정','연휴','나무','설경','시청','나흘','우수','터널','목련',
              '일출','개나리','기운','스카이','연일','성동구','산불','진화','낙시터','평택','매장','대란',
              '동아일보','공기','라며','산책','강종민','축구','프레','심기','작용','카메라','사랑','부처님',
              '송골송골','이물질','개양귀비','구경','바다','시민활동','동급','튤립','해양','모내기','안개',
              '경기장','둥지','지방선거','철쭉','탄천','드론','서귀포','비행','소만','추억','다리','고기압',
              '대진','고송민','거부','활약','시민행동','선언','이의','취소','형형색색','무상','잠실','운행',
              '대기오염','임현정','협약','전망대','한강','식용','군락','환경재단','어린이집','구름','울릉도',
              '연꽃','밀양강','전문가','모두','코스모스','마포구','봄날','대박','중앙','배경','꽁꽁','단풍',
              '눈길','기온','영하','분수','계곡','농장','화성','교양','가득','먼지','단계','고양','고양시',
              '화단','이형기','알피','대학교','성큼','선선','참가자','태현','광진구','평창군','최동준',
              '처서','동민','서툴','해운대해수욕장','해운대','극심','본격','물고기','니스','대관령','상공',
              '전신','서해안','길이','나들','소재','시내','독도','동선','인공','산란','공개','창원시',
              '민주당','이진욱','방문','노동','물이','삼척','구조','통행','밤사이','안산','뉴시스',
              '서울','벚꽃','뉴스','시민','기자','주민','국민','지속','지자체','북적','불법','최대',
              '두번째','세종로','질환','이용','보고','박원순','뿌연','유튜브','모든','주시 반영','주시',
              '반영','아름','더위','기고','아름 다운','도심','다운','국적','고명진','어가','지고','지나',
              '어진','이루','알리','상암동','대공','시작','연출','부터','면서','부터','유의','다가',
              '면서','린다','만들기','마련','전시회','다소','거나','아지','나타','제거','총장','참석',
              '발길','화려','라보','피하','가운','사흠','국적','매달','연출','대공','기고','반영','부터 아지',
              '다가 부터','부터 부터','수요일','화요일','월요일','목요일','금요일','토요일','일요일','유의 바란',
              '박원순','온라인','공식','신청','진행','활용','정도','설명','기간','이후','다른',
              '발표','주장','지적','우려','가능성','정도','요구','과정','한편','동안','다시','상부',
              '이어진','온라인','공식','우려','처음','이유','다시','정도','요구','크게','사실',
              '기간','포함','다음','화보','한편','가능성','이후','설명','분석','지금','각각','발표',
              '크게','동안','이후','처음','주요','내용','실시','어린이','확인','특보','비롯','분포',
              '증가','중인','가량','아시아','이동','지역별','규칙','바로','이하'
             ]
            
            stop_words = list(set(stop_words))

            if(len(s_words) != 0):
                stop_words = s_words
                
            vect = CountVectorizer(stop_words = stop_words,ngram_range=(1, 2),min_df=0.005,max_df=0.3)
       
            X = vect.fit_transform(corpus)

            count = X.toarray().sum(axis=0)
            idx = np.argsort(-count)
            count = count[idx]

            feature_name = np.array(vect.get_feature_names())[idx]
            #plt.bar(range(len(count)), count)
            #plt.show()

            co_word_list = list(zip(feature_name[:num], count[:num]))
            co_word_list = pd.DataFrame(co_word_list)
            filename = "".join(w1)
            co_word_list.to_csv("./doc_level_cowords/" + prefix + filename + "_dcowords" + ".csv",encoding="UTF8")
   
       # 문장레벨 동시 출현 단어
        def sentence_level_coccurrence_words(self,w1, num=100, ma="twitter2",prefix="",s_words=[]):
            
            if ma == "twitter":
                self.morph_analyzer = self.twitter
            if ma == "twitter2":
                self.morph_analyzer = self.twitter2  

            file_list =  self.filtered_idx_result_df['file_list'].tolist()
            filtered_sentences= []
            for i in range(len(file_list)):
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                sentences = self.split_text(doc)
                
                for s in sentences:
                    filtered_sentences.append(s)
            
            self.temp = pd.DataFrame(filtered_sentences)
            
            for w in w1:
                temp = []
                for s in filtered_sentences :
                    p = re.compile(w)
                    ridx = p.search(s)

                    if(ridx == None):
                        continue   
                    s = s.strip()
                    temp.append(s)
                filtered_sentences = temp
            
            self.temp = pd.DataFrame(filtered_sentences)
            print(self.temp)
            # 파일 내용 읽어서 코퍼스에 저장
    
            corpus = []     
            for i in range(len(filtered_sentences)):
                corpus.append(" ".join(self.morph_analyzer.nouns(filtered_sentences[i])))
            
                      # Stop words 
            stop_words = ['광화문','기념','포항','화제','전북','남해','전남','유행','일보','마리','안전','오후','버스',
              '관리','세상','경남','업계','하늘','우산','종로구','하경민','미디어','중부','기자회견','출근',
              '공동','활짝','오전','남부','기술','청주','기사','진주','추진','날씨','집결','계획','대표',
              '밀양시','장관','클릭','공사','전체','판단','전망','함양','관광객','여러분','영상','문의',
              '지정','대체로','경향','억원','경북','지난','대회','제주도','공감','포항시','의원',
              '사진','춘천','군청','평가','산천어','한국','한윤','스님','축제','기상청','최저','추위','동해',
              '규모','행사','적극','함양군','호기','대책','경기','울산','기업','디지털','신문','일본',
              '관계자','최고','코리아','피해','예상','시설','지구','미만','종원','복원','최신','미리','발견',
              '회원','지방','예보','부산','제보','화천','만화','관련','총회','현상','성장','홍보','우리',
              '사업','금지','기준','트렌드','광주','연구','환경','물결','제품','제공','무료','작품','참석자',
              '제주','평년','강원','국제','가치나','가운데','농도','처리','연합','정책','세계','경우',
              '조사','공단','대한','모바일','밀양','공원','지역','국내','주변','서울시','국회','전재',
              '경기도','국가','강릉','위원회','정부','전주','올해','협의','초과','산업','차량','대강','언론',
              '연합뉴스','인근','단체','발령','북부','정보','무단','조성','김용만','대해','아침','대구',
              '결과','태안','안지','환경부','협악','먹이','전국','보이','지원','경향신문','주의보','건설', 
              '경제','자웅','위해','채용','배포','발효','독자','조금','개발','때문','일원','꽃망울','회의',
              '차용','마을','수산','기상','협회','해수욕장','유엔','여수','사회','단지','유치','검사','구간',
              '파란','바람','서해','해상','전시','창원','기름','새만금','반대','골프장','중단','동해안',
              '장미','제도','인증','미투데이','바닷가','설악산','한라산','아파트','해경','충남','월드컵',
              '완연','터트린','맑은','피어','만끽','청명','수준','소방','북상','영동','풍랑','영산강','환경청',
              '일대','정비','꽃샘추위','계정','소통','미국','사람','한겨레','상승','보고서','투표','이자',
              '여의도','윤중로','남산','설천면','서면','밀양아리랑','대전','충북','최강','주가','빅데이터',
              '김선웅','이영환','학생','권역','춘분','스타벅스','커피','청계천','태백산','배훈식','김진아',
              '페이스북','바로가기','트위터','몽골','아들','해제','처분','기해','안성','환자','평균','재판',
              '캠페인','배출','사용','점검','채널','분리','설정','활동','영향','보통','이상','측정','상태', 
              '정취','낙동강','해체','환경운동','인사말','공급','원인','족구','근본','진도','계속','외모',
              '업체','회관','상림','오늘','내일','시세','연구원','지난해','센터','대상','이번','사월','예정', 
              '가장','침대','타워','아래','종록','재촉','시간','파크','대저','스포츠','터트려','농촌진흥청',
              '통해' '차관','이데일리','동물원','이천','경보','소독','인양','가을','장마전선','봄비','겨울',
              '김은경','안병옥','이정선','여신','래시','명동','퍼포먼스','경찰','성분','영서','차차','빙어',
              '세종','인천','함양읍','수원','서초구','중구','송파구','거창군','용인','남면','삼문동','매화',
              '이나','통해','에서','잡고','맺힌','뽐내','채팅','맨손','잡기','평창동계올림픽','긴급','유채꽃',
              '구매','현장','구독','생활','류형','포토','메인','단속','소식','공항','제주시','개막','물질',
              '인제','남구','태백','나들이','강진','김태식','동구','화천군','낚시','우장','속초','사육',
              '개선','시행','문제','차관','방안','주무관','무술년','새해','상춘객','공모전','현재','의성',
              '화천읍','낭만','비상','뿌옇','출근길','도로','대부분','앞바다','조치','매우','고성','차바',
              '새벽','밤새','일부','전날','군산','중심','맑음','주말','보가','임태훈','기록','가족','박주성',
              '체험','청사','발생','산림청','설치','최근','협력','대응','날인','자태','인제군','마지막',
              '식목일','서귀포시','영등포구','성남시','고승민','발걸음','고범준','부제','치가','사거리','거리',
              '롯데','주위','모습','월드','용산구','망신','이하늬','세영','야구','콜라','가드','광장',
              '유혹','첫날','탐방','자락','세종시','맞이','패럴림픽','하루','이동면','경칩','성남','입춘',
              '기승','보기','협약','기관','운영','수거','작업','조명','장병','주차장','올림픽','운항','빗방울',
              '백승','곳곳','방역','하동','하동군','연휴','추석','황금','배병수','홍천','평창','과장',
              '휴식','강릉시','유형','화창','동진','최진','사이','포럼','개최','조경규','국립',
              '운세','후회','여기','직원','물든','대통령','호텔','바닥','북구','청와대','추연','유해',
              '왼쪽','시장','공연','주제','오른쪽','수원시','촬영','주년','정원','청소','현지','여명','강남구',
              '판매','통합','종합','네이버','지청','당분간','추상','햇살','연일','풍경','흥부가','위로',
              '이틀','한경닷컴','유승민','무소속','조현아','노출','총선','스폰서','비키니','치어리더',
              '수확','개방','상황','본부','관측','스탠드','당부','푸른','정상','김경','고현면','고현면',
              '오색','정유년','저녁','서울대','훈련','채취','권현구','김인철','이정섭','미녀','몸짱','에버랜드',
              '매일','서비스','최창호','피해자','병원','저작권','적발','사고','남남서','건물','신고','국제공항',
              '교육청','행동','행동','데이트','확률','가끔','점차','전해','강원도','일이','핫이슈','봄꽃',
              '오픈','서비스','윤성규','성매매','과천','대한항공','몸매','동영상','봉사활동','유강','정선',
              '후보','문호','업데이트','고속도로','태양','물놀이','의상','쯔위','드레스','학교','현황',
              '남남서','동해','내륙','중부지방','분야','폭스바겐','문화','광장','주연','벗꽃','리우','지사',
              '경주','경주시','글로벌','실시간','인턴','후보','특별','형산강','연일읍','북한','마트','가동',
              '한경닷컴','박기량','교육','인사','빗물','안양','그룹','농림축산식품부','이상인','대기',
              '교수','내년','운동장','사막','내외','가장자리','여객선','호인','누드','확산','파문','자료',
              '아나운서','산간','절기','만원','수도권','강수','기상대','주의','람사르','지리산','발전',
              '결항','산수유','북주','외국인','인산인해','주상','부평','둘째','서울시장','지하철','공공기관',
              '자자체','촉구','동남아','새끼','학원','산지','인파','썰매','사무소','휴일','로봇','금은',
              '우리나라','적용','권현구','가정','연휴','나무','설경','시청','나흘','우수','터널','목련',
              '일출','개나리','기운','스카이','연일','성동구','산불','진화','낙시터','평택','매장','대란',
              '동아일보','공기','라며','산책','강종민','축구','프레','심기','작용','카메라','사랑','부처님',
              '송골송골','이물질','개양귀비','구경','바다','시민활동','동급','튤립','해양','모내기','안개',
              '경기장','둥지','지방선거','철쭉','탄천','드론','서귀포','비행','소만','추억','다리','고기압',
              '대진','고송민','거부','활약','시민행동','선언','이의','취소','형형색색','무상','잠실','운행',
              '대기오염','임현정','협약','전망대','한강','식용','군락','환경재단','어린이집','구름','울릉도',
              '연꽃','밀양강','전문가','모두','코스모스','마포구','봄날','대박','중앙','배경','꽁꽁','단풍',
              '눈길','기온','영하','분수','계곡','농장','화성','교양','가득','먼지','단계','고양','고양시',
              '화단','이형기','알피','대학교','성큼','선선','참가자','태현','광진구','평창군','최동준',
              '처서','동민','서툴','해운대해수욕장','해운대','극심','본격','물고기','니스','대관령','상공',
              '전신','서해안','길이','나들','소재','시내','독도','동선','인공','산란','공개','창원시',
              '민주당','이진욱','방문','노동','물이','삼척','구조','통행','밤사이','안산','뉴시스',
              '서울','벚꽃','뉴스','시민','기자','주민','국민','지속','지자체','북적','불법','최대',
              '두번째','세종로','질환','이용','보고','박원순','뿌연','유튜브','모든','주시 반영','주시',
              '반영','아름','더위','기고','아름 다운','도심','다운','국적','고명진','어가','지고','지나',
              '어진','이루','알리','상암동','대공','시작','연출','부터','면서','부터','유의','다가',
              '면서','린다','만들기','마련','전시회','다소','거나','아지','나타','제거','총장','참석',
              '발길','화려','라보','피하','가운','사흠','국적','매달','연출','대공','기고','반영','부터 아지',
              '다가 부터','부터 부터','수요일','화요일','월요일','목요일','금요일','토요일','일요일','유의 바란',
              '박원순','온라인','공식','신청','진행','활용','정도','설명','기간','이후','다른',
              '발표','주장','지적','우려','가능성','정도','요구','과정','한편','동안','다시','상부',
              '이어진','온라인','공식','우려','처음','이유','다시','정도','요구','크게','사실',
              '기간','포함','다음','화보','한편','가능성','이후','설명','분석','지금','각각','발표',
              '크게','동안','이후','처음','주요','내용','실시','어린이','확인','특보','비롯','분포',
              '증가','중인','가량','아시아','이동','지역별','규칙','바로','이하'
             ]
            
            stop_words = list(set(stop_words))

            
            if(len(s_words) != 0):
                stop_words = s_words
                
            vect = CountVectorizer(stop_words = stop_words,ngram_range=(1, 2),min_df=0.005,max_df=0.3)
       
            X = vect.fit_transform(corpus)

            count = X.toarray().sum(axis=0)
            idx = np.argsort(-count)
            count = count[idx]

            feature_name = np.array(vect.get_feature_names())[idx]
 
            co_word_list = list(zip(feature_name[:num], count[:num]))
            co_word_list = pd.DataFrame(co_word_list)
            filename = "".join(w1)
            co_word_list.to_csv("./sentence_level_cowords/" + prefix + filename + "_scowords" + ".csv",encoding="UTF8")
            
        def sentence_summarization(self,cnt=10):
            
            t_list = []
            for s in self.filtered_sentences:
                t= re.sub('[^가-힝0-9a-zA-Z\\s]', '', s)
                t_list.append(t)
                
            p_sentences = ". ".join(t_list)
            textrank = TextRank(b)
            t = textrank.summarize(count=cnt)
            t = a.split("\n")
            
            t = pd.DataFrame(t)
            t.columns=['Sentences']
            result.to_csv("./sentence_summary_result/" + "ss_" + filename + ".csv",encoding="UTF8")
        
        # 명사 인식
        def add_unknown_nouns(self):
            file_list = self.filtered_idx_result_df['file_list'].tolist()
            corpus = [] 
            for i in range(len(file_list)):
                f = open(file_list[i], 'r',encoding='utf-8',errors="ignore")
                doc = f.read()
                doc = doc.replace("\n","")
                corpus.append("".join(doc))

            corpus = pd.DataFrame(corpus)
            corpus.to_csv("./corpus.txt",sep=" ", header=None)
            
            corpus_fname = './corpus.txt'
            sents = DoublespaceLineCorpus(corpus_fname, iter_sent=True)

            noun_extractor = LRNounExtractor_v2(verbose=True)
            noun_extractor.train(sents)
            nouns = noun_extractor.extract()
            nouns = list(nouns.keys())
            self.nouns = nouns
            self.twitter.add_dictionary(nouns, 'Noun')
                
           # nouns
        

In [23]:
# 키워드 기반 기후변화 문서 정제

na1 = NaverNewsAnalyzer()
na1.filtering_date('2005-01-01','2017-12-31')
na1.filtering_contents(na1.climate_words,False)

na2 = NaverNewsAnalyzer()
na2.filtering_date('2005-01-01','2017-12-31')
# 토픽모델링을 통한 기후변화 클러스터 추출
#na2.topic_modeling([5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],prefix = "s1",random_seed=1,ma="twitter2")
na2.load_lda_result("/s3_result_n_18_")
na2.select_topics([2,5,7,11,12])
na = na1
na.combine_idx_result(na2)

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
